In [1]:
import os
import pandas as pd
import glob
import seaborn as sns
import pandas as pd
import datetime as dt

## Creating temporal resolution features

In [2]:
path_input=os.path.join(os.getcwd(), '../data/input/dresden/')
path_output_weather = os.path.join(os.getcwd(), '../data/output/weather/')
path_output=os.path.join(os.getcwd(), '../data/output/')
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv(os.path.join(path_output, "Trips.csv"),index_col=0)

In [4]:
df

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,24_sum,6_sum,2_sum,1_sum,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,265,36,19,8,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff,265,17,7,2019-01-20 00:00:00,47,3,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,265,33,17,17,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff,265,7,2,2019-01-20 00:00:00,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,0,Malterstraße (Haltestelle),0,frame_lock,4483.0,13.691130,7551004130,False,0,True,0,0,2019-01-20 00:33:00,4405670,3,51.028370,Wundtstr. / Zellescher Weg,0,4458.0,13.747730,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,26.0,0.0,265,33,17,17,891f1b55903ffff,891f1b46003ffff,881f1b5591fffff,881f1b4601fffff,265,6,3,2019-01-20 00:00:00,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,265,33,17,17,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff,265,6,1,2019-01-20 00:00:00,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,15,Friedensstraße/Conradstr.,0,analog_code_lock,4373.0,13.741257,0,False,0,True,0,0,2019-01-20 00:35:00,264575,5,51.071740,Friedensstraße/Conradstr.,0,4373.0,13.741257,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,265,33,17,17,891f1b54b97ffff,891f1b54b97ffff,881f1b54b9fffff,881f1b54b9fffff,265,10,7,2019-01-20 00:00:00,47,2,2,2019-01-20 00:00:00,27,2,2,2019-01-20 00:00:00,19,2,2,2019-01-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567434,0,True,0,0,2020-01-20 23:29:00,93908,11248297,3,51.043537,True,71,Tharandter Straße,0,frame_lock,4507.0,13.703979,7551012906,Fal

In [5]:
# MAX: DELETE THIS
df = df.drop(["24_sum", "6_sum", "2_sum", "1_sum",
             "24_demand", "24_demand_hex_small", "24_demand_hex_big", "24_agg_time",
             "6_demand", "6_demand_hex_small", "6_demand_hex_big", "6_agg_time",
             "2_demand", "2_demand_hex_small", "2_demand_hex_big", "2_agg_time",
             "1_demand", "1_demand_hex_small", "1_demand_hex_big", "1_agg_time"], axis=1)
df

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,0,Malterstraße (Haltestelle),0,frame_lock,4483.0,13.691130,7551004130,False,0,True,0,0,2019-01-20 00:33:00,4405670,3,51.028370,Wundtstr. / Zellescher Weg,0,4458.0,13.747730,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,26.0,0.0,891f1b55903ffff,891f1b46003ffff,881f1b5591fffff,881f1b4601fffff
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,15,Friedensstraße/Conradstr.,0,analog_code_lock,4373.0,13.741257,0,False,0,True,0,0,2019-01-20 00:35:00,264575,5,51.071740,Friedensstraße/Conradstr.,0,4373.0,13.741257,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54b97ffff,891f1b54b97ffff,881f1b54b9fffff,881f1b54b9fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567434,0,True,0,0,2020-01-20 23:29:00,93908,11248297,3,51.043537,True,71,Tharandter Straße,0,frame_lock,4507.0,13.703979,7551012906,False,0,True,0,0,2020-01-20 23:46:00,4405670,4,51.028370,Wundtstr. / Zellescher Weg,0,4458.0,13.747730,False,-1.6,91.7,0.0,0.0,0.0,3.3,1,1,20,20,0,0,23,23,20,20,1,False,2020-01-20 00:00:00,17.0,27.0,891f1b55933ffff,891f1b46003ffff,881f1b5593fffff,881f1b4601fffff
567435,0,True,0,0,2020-01-20 23:29:00,93857,310792,6,51.038466,True,71,Bonhoeffer Platz,0,frame_lock,4438.0,13.701679,7551012762,False,0,True,0,0,2020-01-20 23:49:00,10299848,16,51.070350,Alexander-Puschkin-Platz,0,4504.0,13.729830,False,-1.6,91.7,0.0,0.0,0.0,3.3,1,1,20,20,0,0,23,23,20,20,1,False,2020-01-20 00:00:00,20.0,25.0,891f1b5592bffff,891f1b5486fffff,881f1b5593fffff,881f1b5487fffff
567436,0,True,0,0,2020-01-20 23:31:00,93334,264575,4,51.071740,True,71,Friedensstraße/Conradstr.,0,frame_lock,4373.0,13.741257,7551005670,False,0,True,0,0,2020-01-20 23:36:00,121771,5,51.071144,Schauburg,0,4327.0,13.750328,False,-2.4,91.7,0.0,0.0,0.0,3.2,1,1,20,20,0,0,23,23,20,20,1,False,202

In [6]:
#datetime conversion
df = df.sort_values(by=['datetime_start'], ascending=True)
df["datetime_start"] = pd.to_datetime(df["datetime_start"])
df["count"] = 1

### How we want to get temporal resolutions
- add 4 columns for each resolution: 24, 6, 2, one hour
- they display # bookings within given time period

### Function to aggregate datasets if needed

In [7]:
#data by timespan 24H, 1H, 4H, 12H
def setDataset(dataset, temp_resol, columnnamegroupby, functions_dic):
    newDataset = dataset.resample(temp_resol, on = columnnamegroupby).agg(functions_dic)
    return newDataset
#requested resolutions
temp_resol = ['24H', '6H', '2H', '1H']

In [8]:
# setDataset(df,'24H','datetime_start',{"count":"count"})

In [9]:
# #merge from dwd data
# df_full = pd.merge_asof(df_merged.sort_values('datetime_start'), 
#                         df_dwd, left_on='datetime_start', 
#                         right_on='MESS_DATUM', 
#                         tolerance=pd.Timedelta('30 min'),
#                         allow_exact_matches=True, direction='nearest')

In [10]:
# def add_temp_resol(dataset):
#     temp_resol = ['24H', '6H', '2H', '1H']
    
#     for i in temp_resol: 
#         data_temp = dataset
#         data_temp = data_temp.resample(temp_resol, on="datetime_start").agg({"count":"count"})
#         df_full = pd.merge_asof(dataset.sort_values('datetime_start'), 
#                          data_temp, left_on='datetime_start', 
#                          right_on='datetime_start', 
#                          tolerance=pd.Timedelta('24 hours'),
#                          allow_exact_matches=True, direction='nearest')
#     return df_full
        
        


### Creating series with # bookings for given time interval

In [11]:
dataset = df.copy()
print("Number of NaNs:", dataset.isna().sum().sum())

Number of NaNs: 140


In [12]:
#base -> Shifts the base time to calculate from by some time amount
#shifting the base to the middle value of given time interval solved the NaN problem.
#Why? Because of our timedelta, which is half the amount of time of the resampled time interval 
#--> for time interval x we want to sum up all in!
#example: for daily counts aka 24 hours the base will be 12 am, the sum will be from 00:00 am until 23:59 pm

# Counts per time resolution (for whole city)
df_24 = dataset.resample("1D", on="datetime_start").agg({"count":"count"})
df_6 = dataset.resample("6H", on="datetime_start").agg({"count":"count"})
df_2 = dataset.resample("2H", on="datetime_start").agg({"count":"count"})
df_1 = dataset.resample("1H", on="datetime_start").agg({"count":"count"})

# Counts per time resolution & small hexagon (For start_time)
df_24_hex_small = dataset.groupby("h3_hex_small_id_start").resample("1D", on="datetime_start").agg({"count":"count"})
df_6_hex_small = dataset.groupby("h3_hex_small_id_start").resample("6H", on="datetime_start").agg({"count":"count"})
df_2_hex_small = dataset.groupby("h3_hex_small_id_start").resample("2H", on="datetime_start").agg({"count":"count"})
df_1_hex_small = dataset.groupby("h3_hex_small_id_start").resample("1H", on="datetime_start").agg({"count":"count"})

# Counts per time resolution & big hexagon (For start_time)
df_24_hex_big = dataset.groupby("h3_hex_big_id_start").resample("1D", on="datetime_start").agg({"count":"count"})
df_6_hex_big = dataset.groupby("h3_hex_big_id_start").resample("6H", on="datetime_start").agg({"count":"count"})
df_2_hex_big = dataset.groupby("h3_hex_big_id_start").resample("2H", on="datetime_start").agg({"count":"count"})
df_1_hex_big = dataset.groupby("h3_hex_big_id_start").resample("1H", on="datetime_start").agg({"count":"count"})

In [13]:
print("Number of NaNs in df_24:", df_24.isna().sum().sum())
print("Number of NaNs in df_6:", df_6.isna().sum().sum())
print("Number of NaNs in df_2:", df_2.isna().sum().sum())
print("Number of NaNs in df_1:", df_1.isna().sum().sum())

df_24 = df_24[df_24["count"] != 0]
df_6 = df_6[df_6["count"] != 0]
df_2 = df_2[df_2["count"] != 0]
df_1 = df_1[df_1["count"] != 0]

display(df_24,df_6,df_2,df_1)
#all NaN lol

Number of NaNs in df_24: 0
Number of NaNs in df_6: 0
Number of NaNs in df_2: 0
Number of NaNs in df_1: 0


,count
datetime_start,
2019-01-20,265
2019-01-21,529
2019-01-22,477
2019-01-23,530
2019-01-24,1092
...,...
2020-01-16,1520
2020-01-17,1033
2020-01-18,810


,count
datetime_start,
2019-01-20 00:00:00,47
2019-01-20 06:00:00,60
2019-01-20 12:00:00,104
2019-01-20 18:00:00,54
2019-01-21 00:00:00,25
...,...
2020-01-19 18:00:00,199
2020-01-20 00:00:00,33
2020-01-20 06:00:00,317


,count
datetime_start,
2019-01-20 00:00:00,27
2019-01-20 02:00:00,14
2019-01-20 04:00:00,6
2019-01-20 06:00:00,4
2019-01-20 08:00:00,32
...,...
2020-01-20 14:00:00,140
2020-01-20 16:00:00,184
2020-01-20 18:00:00,118


,count
datetime_start,
2019-01-20 00:00:00,19
2019-01-20 01:00:00,8
2019-01-20 02:00:00,9
2019-01-20 03:00:00,5
2019-01-20 04:00:00,3
...,...
2020-01-20 19:00:00,62
2020-01-20 20:00:00,43
2020-01-20 21:00:00,27


### Modify Small Hexagon Aggregate df's
* Kick out entries where no trips took place
* Reset index so hex id and datetime are transformed from indices to callable columns to merge on & by
* Add aggregation time as copy of datetime so it will be automatically added when merging

In [14]:
df_24_hex_small = df_24_hex_small[df_24_hex_small["count"] != 0]
df_24_hex_small.reset_index(inplace=True)
df_24_hex_small["24_agg_time"] = df_24_hex_small["datetime_start"]

df_6_hex_small = df_6_hex_small[df_6_hex_small["count"] != 0]
df_6_hex_small.reset_index(inplace=True)
df_6_hex_small["6_agg_time"] = df_6_hex_small["datetime_start"]

df_2_hex_small = df_2_hex_small[df_2_hex_small["count"] != 0]
df_2_hex_small.reset_index(inplace=True)
df_2_hex_small["2_agg_time"] = df_2_hex_small["datetime_start"]

df_1_hex_small = df_1_hex_small[df_1_hex_small["count"] != 0]
df_1_hex_small.reset_index(inplace=True)
df_1_hex_small["1_agg_time"] = df_1_hex_small["datetime_start"]

display(df_24_hex_small,df_6_hex_small,df_2_hex_small,df_1_hex_small)

,h3_hex_small_id_start,datetime_start,count,24_agg_time
0,891e247b627ffff,2019-01-24,2,2019-01-24
1,891e247b627ffff,2019-02-05,4,2019-02-05
2,891e247b627ffff,2019-02-12,23,2019-02-12
3,891e247b627ffff,2019-02-13,22,2019-02-13
4,891e247b627ffff,2019-02-14,2,2019-02-14
...,...,...,...,...
79484,891fac18ecfffff,2019-05-01,1,2019-05-01
79485,891fac18ecfffff,2019-05-02,1,2019-05-02
79486,891fac19dafffff,2019-05-03,1,2019-05-03
79487,891fac19db3ffff,2019-04-30,3,2019-04-30


,h3_hex_small_id_start,datetime_start,count,6_agg_time
0,891e247b627ffff,2019-01-24 06:00:00,2,2019-01-24 06:00:00
1,891e247b627ffff,2019-02-05 12:00:00,2,2019-02-05 12:00:00
2,891e247b627ffff,2019-02-05 18:00:00,2,2019-02-05 18:00:00
3,891e247b627ffff,2019-02-12 12:00:00,9,2019-02-12 12:00:00
4,891e247b627ffff,2019-02-12 18:00:00,14,2019-02-12 18:00:00
...,...,...,...,...
168916,891fac18ecfffff,2019-05-02 06:00:00,1,2019-05-02 06:00:00
168917,891fac19dafffff,2019-05-03 18:00:00,1,2019-05-03 18:00:00
168918,891fac19db3ffff,2019-04-30 06:00:00,1,2019-04-30 06:00:00
168919,891fac19db3ffff,2019-04-30 12:00:00,2,2019-04-30 12:00:00


,h3_hex_small_id_start,datetime_start,count,2_agg_time
0,891e247b627ffff,2019-01-24 08:00:00,2,2019-01-24 08:00:00
1,891e247b627ffff,2019-02-05 16:00:00,2,2019-02-05 16:00:00
2,891e247b627ffff,2019-02-05 18:00:00,2,2019-02-05 18:00:00
3,891e247b627ffff,2019-02-12 12:00:00,1,2019-02-12 12:00:00
4,891e247b627ffff,2019-02-12 14:00:00,2,2019-02-12 14:00:00
...,...,...,...,...
261194,891fac18ecfffff,2019-05-02 06:00:00,1,2019-05-02 06:00:00
261195,891fac19dafffff,2019-05-03 18:00:00,1,2019-05-03 18:00:00
261196,891fac19db3ffff,2019-04-30 08:00:00,1,2019-04-30 08:00:00
261197,891fac19db3ffff,2019-04-30 12:00:00,2,2019-04-30 12:00:00


,h3_hex_small_id_start,datetime_start,count,1_agg_time
0,891e247b627ffff,2019-01-24 09:00:00,2,2019-01-24 09:00:00
1,891e247b627ffff,2019-02-05 17:00:00,2,2019-02-05 17:00:00
2,891e247b627ffff,2019-02-05 18:00:00,1,2019-02-05 18:00:00
3,891e247b627ffff,2019-02-05 19:00:00,1,2019-02-05 19:00:00
4,891e247b627ffff,2019-02-12 13:00:00,1,2019-02-12 13:00:00
...,...,...,...,...
318954,891fac18ecfffff,2019-05-02 06:00:00,1,2019-05-02 06:00:00
318955,891fac19dafffff,2019-05-03 18:00:00,1,2019-05-03 18:00:00
318956,891fac19db3ffff,2019-04-30 08:00:00,1,2019-04-30 08:00:00
318957,891fac19db3ffff,2019-04-30 13:00:00,2,2019-04-30 13:00:00


### Modify Big Hexagon Aggregate df's
* Kick out entries where no trips took place
* Reset index so hex id and datetime are transformed from indices to callable columns to merge on & by

In [15]:
df_24_hex_big = df_24_hex_big[df_24_hex_big["count"] != 0]
df_24_hex_big.reset_index(inplace=True)

df_6_hex_big = df_6_hex_big[df_6_hex_big["count"] != 0]
df_6_hex_big.reset_index(inplace=True)

df_2_hex_big = df_2_hex_big[df_2_hex_big["count"] != 0]
df_2_hex_big.reset_index(inplace=True)

df_1_hex_big = df_1_hex_big[df_1_hex_big["count"] != 0]
df_1_hex_big.reset_index(inplace=True)

display(df_24_hex_big,df_6_hex_big,df_2_hex_big,df_1_hex_big)

,h3_hex_big_id_start,datetime_start,count
0,881e247b63fffff,2019-01-24,2
1,881e247b63fffff,2019-02-05,4
2,881e247b63fffff,2019-02-12,23
3,881e247b63fffff,2019-02-13,22
4,881e247b63fffff,2019-02-14,2
...,...,...,...
30164,881fac18edfffff,2019-05-01,1
30165,881fac18edfffff,2019-05-02,1
30166,881fac19dbfffff,2019-04-30,3
30167,881fac19dbfffff,2019-05-03,1


,h3_hex_big_id_start,datetime_start,count
0,881e247b63fffff,2019-01-24 06:00:00,2
1,881e247b63fffff,2019-02-05 12:00:00,2
2,881e247b63fffff,2019-02-05 18:00:00,2
3,881e247b63fffff,2019-02-12 12:00:00,9
4,881e247b63fffff,2019-02-12 18:00:00,14
...,...,...,...
78749,881fac18edfffff,2019-05-02 06:00:00,1
78750,881fac19dbfffff,2019-04-30 06:00:00,1
78751,881fac19dbfffff,2019-04-30 12:00:00,2
78752,881fac19dbfffff,2019-05-03 18:00:00,1


,h3_hex_big_id_start,datetime_start,count
0,881e247b63fffff,2019-01-24 08:00:00,2
1,881e247b63fffff,2019-02-05 16:00:00,2
2,881e247b63fffff,2019-02-05 18:00:00,2
3,881e247b63fffff,2019-02-12 12:00:00,1
4,881e247b63fffff,2019-02-12 14:00:00,2
...,...,...,...
147641,881fac18edfffff,2019-05-02 06:00:00,1
147642,881fac19dbfffff,2019-04-30 08:00:00,1
147643,881fac19dbfffff,2019-04-30 12:00:00,2
147644,881fac19dbfffff,2019-05-03 18:00:00,1


,h3_hex_big_id_start,datetime_start,count
0,881e247b63fffff,2019-01-24 09:00:00,2
1,881e247b63fffff,2019-02-05 17:00:00,2
2,881e247b63fffff,2019-02-05 18:00:00,1
3,881e247b63fffff,2019-02-05 19:00:00,1
4,881e247b63fffff,2019-02-12 13:00:00,1
...,...,...,...
205109,881fac18edfffff,2019-05-02 06:00:00,1
205110,881fac19dbfffff,2019-04-30 08:00:00,1
205111,881fac19dbfffff,2019-04-30 13:00:00,2
205112,881fac19dbfffff,2019-05-03 18:00:00,1


### Test into which aggregation time period an example trip will count

In [16]:
dataset[(dataset["datetime_start"] == "2019-09-08 00:00:00")]

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,count
395250,0,False,0,12,2019-09-08,93267,21682928,1,51.088751,True,71,BIKE 93267,0,frame_lock,0.0,13.79986,7551003439,False,0,False,0,12,2019-09-08 01:24:00,21688100,1,51.063769,BIKE 93267,0,0.0,13.748174,False,11.4,91.3,0.0,0.0,0.0,1.2,9,9,8,8,6,6,0,1,251,251,4,True,2019-09-08 00:00:00,84.0,149.0,891f1b094bbffff,891f1b54b07ffff,881f1b094bfffff,881f1b54b1fffff,1


In [17]:
display(df_6_hex_small[(df_6_hex_small["h3_hex_small_id_start"] == "891f1b094bbffff") & (df_6_hex_small["datetime_start"] == "2019-09-07 18:00:00")])
display(df_6_hex_small[(df_6_hex_small["h3_hex_small_id_start"] == "891f1b094bbffff") & (df_6_hex_small["datetime_start"] == "2019-09-08 00:00:00")])
display(df_6_hex_small[(df_6_hex_small["h3_hex_small_id_start"] == "891f1b094bbffff") & (df_6_hex_small["datetime_start"] == "2019-09-08 06:00:00")])

,h3_hex_small_id_start,datetime_start,count,6_agg_time


,h3_hex_small_id_start,datetime_start,count,6_agg_time
11971,891f1b094bbffff,2019-09-08,1,2019-09-08


,h3_hex_small_id_start,datetime_start,count,6_agg_time



### Merge temporal datasets to our Trips.csv
First, define a method which merges every data for one time resolution

In [18]:
def merge_demand_to_trips(df, time_res):
    print("=== Merging demand for time resolution:", time_res, "hours ===")
    if time_res == "24":
        df_time = df_24
        df_hex_small = df_24_hex_small
        df_hex_big = df_24_hex_big
    elif time_res == "6":
        df_time = df_6
        df_hex_small = df_6_hex_small
        df_hex_big = df_6_hex_big
    elif time_res == "2":
        df_time = df_2
        df_hex_small = df_2_hex_small
        df_hex_big = df_2_hex_big
    elif time_res == "1":
        df_time = df_1
        df_hex_small = df_1_hex_small
        df_hex_big = df_1_hex_big
    else:
        print("Invalid time resolution given. Pick one of: 24, 6, 2, 1.")
        return df
    
    # demand per time resolution
    print("Merging demand for whole city...")
    df_full = pd.merge_asof(df.sort_values('datetime_start'), 
                            df_time,
                            left_on='datetime_start',
                            right_on='datetime_start',
                            allow_exact_matches=True,
                            direction='backward')
    df_full.rename(columns={'count':time_res+'_demand',},
                    inplace=True)
    
    # demand per time resolution and big hexagon
    print("Merging demand for big hexagons...")
    df_full = pd.merge_asof(df_full.sort_values('datetime_start'), 
                            df_hex_big.sort_values("datetime_start"),
                            left_on='datetime_start', 
                            right_on='datetime_start',
                            by="h3_hex_big_id_start",
                            allow_exact_matches=True,
                            direction='backward')
    df_full.rename(columns={'count':time_res+'_demand_hex_big'}, 
                    inplace=True)
    
    # demand per time resolution and small hexagon
    print("Merging demand for small hexagons...")
    df_full = pd.merge_asof(df_full.sort_values('datetime_start'), 
                            df_hex_small.sort_values("datetime_start"),
                            left_on='datetime_start', 
                            right_on='datetime_start',
                            by="h3_hex_small_id_start",
                            allow_exact_matches=True,
                            direction='backward')
    df_full.rename(columns={'count':time_res+'_demand_hex_small'}, 
                    inplace=True)
    
    print()
    return df_full

Delete count column as it was only a placeholder for temporal dataset calculation

In [19]:
df_full = dataset.drop("count", axis=1)

Run defined method with all time resolutions & show result

In [20]:
time_resolutions = ["24", "6", "2", "1"]

for time in time_resolutions:
    df_full = merge_demand_to_trips(df=df_full, time_res=time)

=== Merging demand for time resolution: 24 hours ===
Merging demand for whole city...
Merging demand for big hexagons...
Merging demand for small hexagons...

=== Merging demand for time resolution: 6 hours ===
Merging demand for whole city...
Merging demand for big hexagons...
Merging demand for small hexagons...

=== Merging demand for time resolution: 2 hours ===
Merging demand for whole city...
Merging demand for big hexagons...
Merging demand for small hexagons...

=== Merging demand for time resolution: 1 hours ===
Merging demand for whole city...
Merging demand for big hexagons...
Merging demand for small hexagons...



In [21]:
df_full

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff,265,17,7,2019-01-20,47,3,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff,265,7,2,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,0,Malterstraße (Haltestelle),0,frame_lock,4483.0,13.691130,7551004130,False,0,True,0,0,2019-01-20 00:33:00,4405670,3,51.028370,Wundtstr. / Zellescher Weg,0,4458.0,13.747730,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,26.0,0.0,891f1b55903ffff,891f1b46003ffff,881f1b5591fffff,881f1b4601fffff,265,6,3,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff,265,6,1,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,15,Friedensstraße/Conradstr.,0,analog_code_lock,4373.0,13.741257,0,False,0,True,0,0,2019-01-20 00:35:00,264575,5,51.071740,Friedensstraße/Conradstr.,0,4373.0,13.741257,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54b97ffff,891f1b54b97ffff,881f1b54b9fffff,881f1b54b9fffff,265,10,7,2019-01-20,47,2,2,2019-01-20 00:00:00,27,2,2,2019-01-20 00:00:00,19,2,2,2019-01-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567434,0,True,0,0,2020-01-20 23:29:00,93857,310792,6,51.038466,True,71,Bonhoeffer Platz,0,frame_lock,4438.0,13.701679,7551012762,False,0,True,0,0,2020-01-20 23:49:00,10299848,16,51.070350,Alexander-Puschkin-Platz,0,4504.0,13.729830,False,-1.6,91.7,0.0,0.0,0.0,3.3,1,1,20,20,0,0,23,23,2

In [22]:
df_show = df_full[["datetime_start",
                 "h3_hex_small_id_start",
                 "h3_hex_big_id_start",
                 "24_demand",
                 "24_demand_hex_big",
                 "24_demand_hex_small",
                 "24_agg_time",
                 "6_demand",
                 "6_demand_hex_big",
                 "6_demand_hex_small",
                 "6_agg_time",
                 "2_demand",
                 "2_demand_hex_big",
                 "2_demand_hex_small",
                 "2_agg_time",
                 "1_demand",
                 "1_demand_hex_big",
                 "1_demand_hex_small",
                 "1_agg_time"]]

print()
print("Number of NaNs:", df_show.isna().sum().sum())
print("Show rows with NaN values:")
display(df_show[df_show.isna().any(axis=1)])
print()
print("Show head of result:")
display(df_show.head(20))


Number of NaNs: 0
Show rows with NaN values:


,datetime_start,h3_hex_small_id_start,h3_hex_big_id_start,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time



Show head of result:


,datetime_start,h3_hex_small_id_start,h3_hex_big_id_start,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time
0,2019-01-20 00:00:00,891f1b54bb3ffff,881f1b54bbfffff,265,17,7,2019-01-20,47,3,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00
1,2019-01-20 00:05:00,891f1b46097ffff,881f1b4609fffff,265,7,2,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00
2,2019-01-20 00:07:00,891f1b55903ffff,881f1b5591fffff,265,6,3,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00
3,2019-01-20 00:07:00,891f1b5591bffff,881f1b5591fffff,265,6,1,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00
4,2019-01-20 00:07:00,891f1b54b97ffff,881f1b54b9fffff,265,10,7,2019-01-20,47,2,2,2019-01-20,27,2,2,2019-01-20,19,2,2,2019-01-20 00:00:00
5,2019-01-20 00:21:00,891f1b4612bffff,881f1b4613fffff,265,1,1,2019-01-20,47,1,1,2019-01-20,27,1,1,2019-01-20,19,1,1,2019-01-20 00:00:00
6,2019-01-20 00:26:00,891f1b46613ffff,881f1b4661fffff,265,1,1,2019-01-20,47,1,1,2019-01-20,27,1,1,2019-01-20,19,1,1,2019-01-20 00:00:00
7,2019-01-20 00:29:00,891f1b46527ffff,881f1b4653fffff,265,5,4,2019-01-20,47,1,1,2019-01-20,27,1,1,2019-01-20,19,1,1,2019-01-20 00:00:00
8,2019-01-20 00:31:00,891f1b54b0bffff,881f1b54b1fffff,265,13,2,2019-01-20,47,6,2,2019-01-20,27,3,1,2019-01-20,19,1,1,2019-01-20 00:00:00
9,2019-01-20 00:33:00,891f1b466b7ffff,881f1b466bfffff,265,12,5,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,1,1,2019-01-20 00:00:00


### Counts based on end time

In [58]:
#datetime conversion
df_full = df_full.sort_values(by=['datetime_end'], ascending=True)
df_full["datetime_end"] = pd.to_datetime(df_full["datetime_end"])
df_full["count"] = 1

In [59]:
dataset_end = df_full.copy()
print("Number of NaNs:", dataset_end.isna().sum().sum())

Number of NaNs: 140


In [60]:
# Counts based on end time
# whole city
df_24_end = dataset_end.resample("1D", on="datetime_end").agg({"count":"count"})
df_6_end = dataset_end.resample("6H", on="datetime_end").agg({"count":"count"})
# small hexagons
df_24_hex_small_end = dataset_end.groupby("h3_hex_small_id_end").resample("1D", on="datetime_end", label="left").agg({"count":"count"})
df_6_hex_small_end = dataset_end.groupby("h3_hex_small_id_end").resample("6H", on="datetime_end", label="left").agg({"count":"count"})
# big hexagons
df_24_hex_big_end = dataset_end.groupby("h3_hex_big_id_end").resample("1D", on="datetime_end", label="left").agg({"count":"count"})
df_6_hex_big_end = dataset_end.groupby("h3_hex_big_id_end").resample("6H", on="datetime_end", label="left").agg({"count":"count"})

In [61]:
print("Number of NaNs in df_24:", df_24_end.isna().sum().sum())
print("Number of NaNs in df_6:", df_6_end.isna().sum().sum())

df_24_end = df_24_end[df_24_end["count"] != 0]
df_6_end = df_6_end[df_6_end["count"] != 0]

display(df_24_end,df_6_end)

Number of NaNs in df_24: 0
Number of NaNs in df_6: 0


,count
datetime_end,
2019-01-20,265
2019-01-21,529
2019-01-22,477
2019-01-23,530
2019-01-24,1092
...,...
2020-01-16,1520
2020-01-17,1033
2020-01-18,810


,count
datetime_end,
2019-01-20 00:00:00,45
2019-01-20 06:00:00,52
2019-01-20 12:00:00,100
2019-01-20 18:00:00,68
2019-01-21 00:00:00,12
...,...
2020-01-19 18:00:00,209
2020-01-20 00:00:00,33
2020-01-20 06:00:00,299


In [62]:
print("Number of NaNs in df_24:", df_24_hex_small_end.isna().sum().sum())
print("Number of NaNs in df_6:", df_6_hex_small_end.isna().sum().sum())

df_24_hex_small_end = df_24_hex_small_end[df_24_hex_small_end["count"] != 0]
df_24_hex_small_end.reset_index(inplace=True)
df_24_hex_small_end["24_agg_time_end"] = df_24_hex_small_end["datetime_end"]

df_6_hex_small_end = df_6_hex_small_end[df_6_hex_small_end["count"] != 0]
df_6_hex_small_end.reset_index(inplace=True)
df_6_hex_small_end["6_agg_time_end"] = df_6_hex_small_end["datetime_end"]

display(df_24_hex_small_end,df_6_hex_small_end)

Number of NaNs in df_24: 0
Number of NaNs in df_6: 0


,h3_hex_small_id_end,datetime_end,count,24_agg_time_end
0,891e247b627ffff,2019-01-24,2,2019-01-24
1,891e247b627ffff,2019-02-05,4,2019-02-05
2,891e247b627ffff,2019-02-12,23,2019-02-12
3,891e247b627ffff,2019-02-13,22,2019-02-13
4,891e247b627ffff,2019-02-14,2,2019-02-14
...,...,...,...,...
82618,891fac19db3ffff,2019-04-30,2,2019-04-30
82619,891fac19db3ffff,2019-05-04,1,2019-05-04
82620,892a992288fffff,2019-09-05,1,2019-09-05
82621,892b8d096bbffff,2019-02-14,1,2019-02-14


,h3_hex_small_id_end,datetime_end,count,6_agg_time_end
0,891e247b627ffff,2019-01-24 06:00:00,2,2019-01-24 06:00:00
1,891e247b627ffff,2019-02-05 12:00:00,2,2019-02-05 12:00:00
2,891e247b627ffff,2019-02-05 18:00:00,2,2019-02-05 18:00:00
3,891e247b627ffff,2019-02-12 12:00:00,9,2019-02-12 12:00:00
4,891e247b627ffff,2019-02-12 18:00:00,14,2019-02-12 18:00:00
...,...,...,...,...
171843,891fac19db3ffff,2019-04-30 12:00:00,1,2019-04-30 12:00:00
171844,891fac19db3ffff,2019-05-04 12:00:00,1,2019-05-04 12:00:00
171845,892a992288fffff,2019-09-05 18:00:00,1,2019-09-05 18:00:00
171846,892b8d096bbffff,2019-02-14 06:00:00,1,2019-02-14 06:00:00


In [63]:
print("Number of NaNs in df_24:", df_24_hex_big_end.isna().sum().sum())
print("Number of NaNs in df_6:", df_6_hex_big_end.isna().sum().sum())

df_24_hex_big_end = df_24_hex_big_end[df_24_hex_big_end["count"] != 0]
df_24_hex_big_end.reset_index(inplace=True)

df_6_hex_big_end = df_6_hex_big_end[df_6_hex_big_end["count"] != 0]
df_6_hex_big_end.reset_index(inplace=True)

display(df_24_hex_big_end,df_6_hex_big_end)

Number of NaNs in df_24: 0
Number of NaNs in df_6: 0


,h3_hex_big_id_end,datetime_end,count
0,881e247b63fffff,2019-01-24,2
1,881e247b63fffff,2019-02-05,4
2,881e247b63fffff,2019-02-12,23
3,881e247b63fffff,2019-02-13,22
4,881e247b63fffff,2019-02-14,2
...,...,...,...
30735,881fac19dbfffff,2019-05-03,1
30736,881fac19dbfffff,2019-05-04,1
30737,882a992289fffff,2019-09-05,1
30738,882b8d096bfffff,2019-02-14,1


,h3_hex_big_id_end,datetime_end,count
0,881e247b63fffff,2019-01-24 06:00:00,2
1,881e247b63fffff,2019-02-05 12:00:00,2
2,881e247b63fffff,2019-02-05 18:00:00,2
3,881e247b63fffff,2019-02-12 12:00:00,9
4,881e247b63fffff,2019-02-12 18:00:00,14
...,...,...,...
79473,881fac19dbfffff,2019-05-03 12:00:00,1
79474,881fac19dbfffff,2019-05-04 12:00:00,1
79475,882a992289fffff,2019-09-05 18:00:00,1
79476,882b8d096bfffff,2019-02-14 06:00:00,1


In [64]:
display(df_6_hex_small_end[(df_6_hex_small_end["h3_hex_small_id_end"] == "891f1b4646fffff") & (df_6_hex_small_end["datetime_end"] == "2019-01-20 00:00:00")])
display(df_6_hex_small_end[(df_6_hex_small_end["h3_hex_small_id_end"] == "891f1b4646fffff") & (df_6_hex_small_end["datetime_end"] == "2019-01-20 01:00:00")])
display(df_6_hex_small_end[(df_6_hex_small_end["h3_hex_small_id_end"] == "891f1b4646fffff") & (df_6_hex_small_end["datetime_end"] == "2019-01-20 02:00:00")])

,h3_hex_small_id_end,datetime_end,count,6_agg_time_end
60077,891f1b4646fffff,2019-01-20,3,2019-01-20


,h3_hex_small_id_end,datetime_end,count,6_agg_time_end


,h3_hex_small_id_end,datetime_end,count,6_agg_time_end


In [65]:
dataset_end[(dataset_end["datetime_end"] > "2019-01-20 00:00:00") & (dataset_end["datetime_end"] < "2019-01-20 02:00:00")]
# dataset_end[dataset_end["h3_hex_small_id_end"] == "891f1b4646fffff"]

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,count
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff,265,6,1,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00,1
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff,265,7,2,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00,1
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff,265,17,7,2019-01-20,47,3,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00,1
6,0,False,0,12,2019-01-20 00:26:00,93478,12098942,1,51.028163,False,0,BIKE 93478,0,analog_code_lock,0.0,13.710755,1206,False,0,True,0,0,2019-01-20 00:28:00,10299678,1,51.023416,Räcknitzhöhe,0,4491.0,13.742453,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46613ffff,891f1b46047ffff,881f1b4661fffff,881f1b4605fffff,265,1,1,2019-01-20,47,1,1,2019-01-20,27,1,1,2019-01-20,19,1,1,2019-01-20 00:00:00,1
7,0,True,0,0,2019-01-20 00:29:00,93577,264599,1,51.044740,False,15,Mosenstraße,0,analog_code_lock,4392.0,13.777725,0,False,0,True,0,0,2019-01-20 00:31:00,264599,1,51.044740,Mosenstraße,0,4392.0,13.777725,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46527ffff,891f1b46527ffff,881f1b4653fffff,881f1b4653fffff,265,5,4,2019-01-20,47,1,1,2019-01-20,27,1,1,2019-01-20,19,1,1,2019-01-20 00:00:00,1
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,0,Malterstraße (Haltestelle),0,frame_lock,4483.0,13.691130,7551004130,False,0,True,0,0,2019-01-20 00:33:00,4405670,3,51.028370,Wundtstr. / Zellescher Weg,0,4458.0,13.747730,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,26.0,0.0,891f1b55903ffff,891f1b46003ffff,881f1b5591fffff,881f1b4601fffff,265,6,3,2019-01-20,47,2,1,2019-01-20,27,2,1,2019-01-20,19,2,1,2019-01-20 00:00:00,1
5,0,True,0,0,2019-01-20 00:21:00,93660,45444,1,51.025741,False,15,Panometer,0,analog_code_lock,4310.0,13.789338,0,False,0,True,0,0,2019-01-20 00:33:00,264532,1,51.034938,Altenberger Platz,0,4336.0,13.807733,False,-3.3,75.3,0.0,0.0,0.

In [66]:
def merge_end_demand(_df, time_res):
    print("=== Merging END demand for time resolution:", time_res, "hours ===")
    if time_res == "24":
        df_time = df_24_end
        df_hex_small = df_24_hex_small_end
        df_hex_big = df_24_hex_big_end
    elif time_res == "6":
        df_time = df_6_end
        df_hex_small = df_6_hex_small_end
        df_hex_big = df_6_hex_big_end
    else:
        print("Invalid time resolution given. Pick one of: 24, 6.")
        return _df
    
    # demand per time resolution
    print("END: Merging demand for whole city...")
    df_full_end = pd.merge_asof(_df.sort_values('datetime_end'), 
                            df_time.sort_values("datetime_end"),
                            left_on='datetime_end',
                            right_on='datetime_end',
                            allow_exact_matches=True,
                            direction='backward')
    df_full_end.rename(columns={'count':time_res+'_demand_end'},
                    inplace=True)
    
    # demand per time resolution and big hexagon
    print("END: Merging demand for big hexagons...")
    df_full_end = pd.merge_asof(df_full_end.sort_values('datetime_end'), 
                            df_hex_big.sort_values("datetime_end"),
                            left_on='datetime_end', 
                            right_on='datetime_end',
                            by="h3_hex_big_id_end",
                            allow_exact_matches=True,
                            direction='backward')
    df_full_end.rename(columns={'count':time_res+'_demand_hex_big_end'}, 
                    inplace=True)
    
    # demand per time resolution and small hexagon
    print("END: Merging demand for small hexagons...")
    df_full_end = pd.merge_asof(df_full_end.sort_values('datetime_end'), 
                            df_hex_small.sort_values("datetime_end"),
                            left_on='datetime_end', 
                            right_on='datetime_end',
                            by="h3_hex_small_id_end",
                            allow_exact_matches=True,
                            direction='backward')
    df_full_end.rename(columns={'count':time_res+'_demand_hex_small_end'}, 
                    inplace=True)
    
    print()
    return df_full_end

In [67]:
df_full_end = dataset_end.drop("count", axis=1)

In [68]:
df_full_end = merge_end_demand(_df=df_full_end, time_res="24")
df_full_end = merge_end_demand(_df=df_full_end, time_res="6")
print("Done")

=== Merging END demand for time resolution: 24 hours ===
END: Merging demand for whole city...
END: Merging demand for big hexagons...
END: Merging demand for small hexagons...

=== Merging END demand for time resolution: 6 hours ===
END: Merging demand for whole city...
END: Merging demand for big hexagons...
END: Merging demand for small hexagons...

Done


In [69]:
df_full_end

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end
0,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff,265,6,1,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,4,1,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff,265,7,2,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,7,7,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
2,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff,265,17,7,2019-01-20,47,3,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,1,3,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
3,0,False,0,12,2019-01-20 00:26:00,93478,12098942,1,51.028163,False,0,BIKE 93478,0,analog_code_lock,0.0,13.710755,1206,False,0,True,0,0,2019-01-20 00:28:00,10299678,1,51.023416,Räcknitzhöhe,0,4491.0,13.742453,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46613ffff,891f1b46047ffff,881f1b4661fffff,881f1b4605fffff,265,1,1,2019-01-20,47,1,1,2019-01-20 00:00:00,27,1,1,2019-01-20 00:00:00,19,1,1,2019-01-20 00:00:00,265,2,2,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
4,0,True,0,0,2019-01-20 00:29:00,93577,264599,1,51.044740,False,15,Mosenstraße,0,analog_code_lock,4392.0,13.777725,0,False,0,True,0,0,2019-01-20 00:31:00,264599,1,51.044740,Mosenstraße,0,4392.0,13.777725,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46527ffff,891f1b46527ffff,881f1b4653fffff,881f1b4653fffff,265,5,4,2019-01-20,47,1,1,2019-01-20 00:00:00,27,1,1,2019-01-20 00:00:00,19,1,1,2019-01-20 00:00:00,265,6,4,2019-01-20,45,2.0,2.0,2019-01-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [70]:
df_show_end = df_full_end[["datetime_end",
                 "h3_hex_small_id_end",
                 "h3_hex_big_id_end",
                 "24_demand_end",
                 "24_demand_hex_big_end",
                 "24_demand_hex_small_end",
                 "24_agg_time_end",
                 "6_demand_end",
                 "6_demand_hex_big_end",
                 "6_demand_hex_small_end",
                 "6_agg_time_end"]]

print()
print("Number of NaNs:", df_show_end.isna().sum().sum())
print("Show rows with NaN values:")
display(df_show_end[df_show_end.isna().any(axis=1)])
print()
print("Show head of result:")
display(df_show_end.head(20))


Number of NaNs: 75
Show rows with NaN values:


,datetime_end,h3_hex_small_id_end,h3_hex_big_id_end,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end
44,2019-01-20 05:48:00,891f1b4658bffff,881f1b4659fffff,265,5,3,2019-01-20,45,NaN,NaN,NaT
96,2019-01-20 11:55:00,891f1b4648bffff,881f1b4649fffff,265,4,2,2019-01-20,52,1.0,NaN,NaT
190,2019-01-20 17:42:00,891f1b09647ffff,881f1b0965fffff,265,10,1,2019-01-20,100,4.0,NaN,NaT
191,2019-01-20 17:43:00,891f1b464b3ffff,881f1b464bfffff,265,9,2,2019-01-20,100,1.0,NaN,NaT
195,2019-01-20 17:51:00,891f1b54b33ffff,881f1b54b3fffff,265,6,1,2019-01-20,100,2.0,NaN,NaT
406,2019-01-21 11:57:00,891f1b4609bffff,881f1b4609fffff,529,19,2,2019-01-21,134,1.0,NaN,NaT
2550,2019-01-24 11:52:00,891f1b461bbffff,881f1b461bfffff,1092,5,1,2019-01-24,739,4.0,NaN,NaT
4297,2019-01-28 11:49:00,891f1b46543ffff,881f1b4655fffff,703,1,1,2019-01-28,214,1.0,NaN,NaT
7020,2019-02-01 17:30:00,891f1b54a5bffff,881f1b54a5fffff,644,1,1,2019-02-01,242,NaN,NaN,NaT
7446,2019-02-02 17:04:00,891f1b0903bffff,881f1b0903fffff,394,2,2,2019-02-02,88,NaN,NaN,NaT



Show head of result:


,datetime_end,h3_hex_small_id_end,h3_hex_big_id_end,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end
0,2019-01-20 00:09:00,891f1b5591bffff,881f1b5591fffff,265,4,1,2019-01-20,45,1.0,1.0,2019-01-20
1,2019-01-20 00:15:00,891f1b465dbffff,881f1b465dfffff,265,7,7,2019-01-20,45,1.0,1.0,2019-01-20
2,2019-01-20 00:28:00,891f1b5598fffff,881f1b5599fffff,265,1,3,2019-01-20,45,1.0,1.0,2019-01-20
3,2019-01-20 00:28:00,891f1b46047ffff,881f1b4605fffff,265,2,2,2019-01-20,45,1.0,1.0,2019-01-20
4,2019-01-20 00:31:00,891f1b46527ffff,881f1b4653fffff,265,6,4,2019-01-20,45,2.0,2.0,2019-01-20
5,2019-01-20 00:33:00,891f1b46003ffff,881f1b4601fffff,265,16,14,2019-01-20,45,6.0,6.0,2019-01-20
6,2019-01-20 00:33:00,891f1b46cc7ffff,881f1b46cdfffff,265,1,1,2019-01-20,45,1.0,1.0,2019-01-20
7,2019-01-20 00:35:00,891f1b54b97ffff,881f1b54b9fffff,265,12,6,2019-01-20,45,1.0,1.0,2019-01-20
8,2019-01-20 00:45:00,891f1b46003ffff,881f1b4601fffff,265,16,14,2019-01-20,45,6.0,6.0,2019-01-20
9,2019-01-20 00:49:00,891f1b46003ffff,881f1b4601fffff,265,16,14,2019-01-20,45,6.0,6.0,2019-01-20


### Analyze NaNs / no NaNs 

In [71]:
#this did not work as expected
#logically, 24_sum should never be nan, while 1_sum could be nan quite often.
df_full.isna().sum()
# 24_sum                   699
# 6_sum                     75
# 2_sum                     23
# 1_sum                      3
#df_full
df_full_end.isna().sum()

p_bike_racks_start         0
p_spot_start               0
p_booked_bikes_start       0
p_place_type_start         0
datetime_start             0
                          ..
24_agg_time_end            0
6_demand_end               0
6_demand_hex_big_end       9
6_demand_hex_small_end    33
6_agg_time_end            33
Length: 80, dtype: int64

In [73]:
#the merge did not work for 20th of January, 2020! 
#this might be, because no matching datetime_start was found
df = df_full_end

df_show = df[df['6_demand_hex_big_end'].isnull()]
df_show = df_show[['datetime_end','h3_hex_big_id_end','24_demand_hex_big_end','6_demand_hex_big_end']]
display(df_show)
# print(df_show.to_string())

#Whats the reason? maybe merge direction 'nearest' is bad for this task? --> NOPE
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html

# A “backward” search selects the last row in the right DataFrame --> 300.000 NaNs
# whose ‘on’ key is less than or equal to the left’s key.

# A “forward” search selects the first row in the right DataFrame --> 200.000 NaNs
# whose ‘on’ key is greater than or equal to the left’s key.

# A “nearest” search selects the row in the right DataFrame --> 700 NaNs
# whose ‘on’ key is closest in absolute distance to the left’s key

#Maybe resampling method is not entirely correct?
#https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.resample.html

,datetime_end,h3_hex_big_id_end,24_demand_hex_big_end,6_demand_hex_big_end
44,2019-01-20 05:48:00,881f1b4659fffff,5,NaN
7020,2019-02-01 17:30:00,881f1b54a5fffff,1,NaN
7446,2019-02-02 17:04:00,881f1b0903fffff,2,NaN
110046,2019-04-20 11:29:00,881f1b4453fffff,1,NaN
184081,2019-05-30 17:59:00,881f1b46b3fffff,1,NaN
311456,2019-07-26 11:39:00,881f1b40b5fffff,1,NaN
311476,2019-07-26 11:51:00,881f1b48dbfffff,1,NaN
348957,2019-08-13 11:49:00,881f1b71a3fffff,1,NaN
363561,2019-08-21 11:41:00,881f1b19a9fffff,1,NaN


In [74]:
df[df["datetime_end"] == "2019-01-20 12:56:00"]

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end
114,0,False,0,12,2019-01-20 12:43:00,93582,12071610,1,51.062906,False,15,BIKE 93582,0,analog_code_lock,0.0,13.723488,10908,False,0,False,0,12,2019-01-20 12:56:00,12111464,1,51.056678,BIKE 93582,0,0.0,13.725931,False,1.5,56.7,0.0,0.0,59.5,1.7,1,1,20,20,6,6,12,12,20,20,1,True,2019-01-20 00:00:00,13.0,0.0,891f1b54a37ffff,891f1b54a2fffff,881f1b54a3fffff,881f1b54a3fffff,265,3,1,2019-01-20,104,2,1,2019-01-20 12:00:00,37,2,1,2019-01-20 12:00:00,13,2,1,2019-01-20 12:00:00,265,1,1,2019-01-20,100,1.0,1.0,2019-01-20 12:00:00


In [75]:
df_6_hex_big_end[df_6_hex_big_end["datetime_end"] == "2019-01-20 13:00:00"]

,h3_hex_big_id_end,datetime_end,count


### Check if timedeltas caused any issues / no issues

In [79]:
df_full_end[df_full_end['24_agg_time']=='2020-01-21']
#keine verschiebung der booking sums

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end


In [80]:
df_full_end[df_full_end['24_agg_time']=='2019-01-19']
#keine verschiebung der booking sums

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end


In [81]:
df_full_end.head(100)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,b_bike_type_start,p_name_start,p_free_racks_start,b_lock_types_start,p_number_start,p_lng_start,b_boardcomputer_start,p_maintenance_start,p_bike_racks_end,p_spot_end,p_booked_bikes_end,p_place_type_end,datetime_end,p_uid_end,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_number_end,p_lng_end,p_maintenance_end,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,month_start,month_end,day_start,day_end,day_of_week_start,day_of_week_end,hour_start,hour_end,day_of_year_start,day_of_year_end,season,weekend,booking_date_start,trip_duration,idle_time,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end,24_demand,24_demand_hex_big,24_demand_hex_small,24_agg_time,6_demand,6_demand_hex_big,6_demand_hex_small,6_agg_time,2_demand,2_demand_hex_big,2_demand_hex_small,2_agg_time,1_demand,1_demand_hex_big,1_demand_hex_small,1_agg_time,24_demand_end,24_demand_hex_big_end,24_demand_hex_small_end,24_agg_time_end,6_demand_end,6_demand_hex_big_end,6_demand_hex_small_end,6_agg_time_end
0,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,0,BIKE 93322,0,analog_code_lock,0.0,13.689620,1265,False,0,False,0,12,2019-01-20 00:09:00,12098234,1,51.041798,BIKE 93322,0,0.0,13.689620,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b5591bffff,891f1b5591bffff,881f1b5591fffff,881f1b5591fffff,265,6,1,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,4,1,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,15,Wohnheim Gret-Palucca.Straße / Lenneplatz,0,analog_code_lock,4486.0,13.744712,0,False,0,False,0,12,2019-01-20 00:15:00,12099344,1,51.049069,BIKE 93576,0,0.0,13.744122,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,10.0,0.0,891f1b46097ffff,891f1b465dbffff,881f1b4609fffff,881f1b465dfffff,265,7,2,2019-01-20,47,2,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,7,7,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
2,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,15,BIKE 93771,0,analog_code_lock,0.0,13.750380,22532,False,0,False,0,12,2019-01-20 00:28:00,12099518,1,51.046234,BIKE 93771,0,0.0,13.693052,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,28.0,0.0,891f1b54bb3ffff,891f1b5598fffff,881f1b54bbfffff,881f1b5599fffff,265,17,7,2019-01-20,47,3,1,2019-01-20 00:00:00,27,2,1,2019-01-20 00:00:00,19,2,1,2019-01-20 00:00:00,265,1,3,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
3,0,False,0,12,2019-01-20 00:26:00,93478,12098942,1,51.028163,False,0,BIKE 93478,0,analog_code_lock,0.0,13.710755,1206,False,0,True,0,0,2019-01-20 00:28:00,10299678,1,51.023416,Räcknitzhöhe,0,4491.0,13.742453,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46613ffff,891f1b46047ffff,881f1b4661fffff,881f1b4605fffff,265,1,1,2019-01-20,47,1,1,2019-01-20 00:00:00,27,1,1,2019-01-20 00:00:00,19,1,1,2019-01-20 00:00:00,265,2,2,2019-01-20,45,1.0,1.0,2019-01-20 00:00:00
4,0,True,0,0,2019-01-20 00:29:00,93577,264599,1,51.044740,False,15,Mosenstraße,0,analog_code_lock,4392.0,13.777725,0,False,0,True,0,0,2019-01-20 00:31:00,264599,1,51.044740,Mosenstraße,0,4392.0,13.777725,False,-3.3,75.3,0.0,0.0,0.0,4.3,1,1,20,20,6,6,0,0,20,20,1,True,2019-01-20 00:00:00,2.0,0.0,891f1b46527ffff,891f1b46527ffff,881f1b4653fffff,881f1b4653fffff,265,5,4,2019-01-20,47,1,1,2019-01-20 00:00:00,27,1,1,2019-01-20 00:00:00,19,1,1,2019-01-20 00:00:00,265,6,4,2019-01-20,45,2.0,2.0,2019-01-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

### Save updated Trips.csv

In [82]:
df_full_end.to_csv(os.path.join(path_output, "Trips.csv"))